<a href="https://colab.research.google.com/github/aiishaa/Sentiment-analysis/blob/main/Sentiment%20Analysis%20with%20Naive%20Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. Here i use BoW + Naive Bayes classifier.

Keras has a built-in IMDb movie reviews dataset that we can use

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 11 19:48:30 2022

@author: admin
"""

# Load in training and test data (note the difference in convention compared to scikit-learn)
from keras.datasets import imdb
import numpy as np
import pandas as pd

(X_train, y_train), (X_test, y_test) = imdb.load_data()

In [ ]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

# take only 15000 document from training set and 2000 from test set to reduce the computing time 
X_train = X_train[:15000]
y_train = y_train[:15000]
X_test = X_test[:2000]
y_test = y_test[:2000]

--- Review ---
[1, 4, 14906, 716, 4, 65, 7, 4, 689, 4367, 6308, 2343, 4804, 28674, 84206, 5270, 32099, 2315, 71688, 12572, 24785, 43394, 4, 10993, 628, 7685, 37, 9, 150, 4, 9820, 4069, 11, 2909, 4, 16287, 847, 313, 6, 176, 63860, 9, 6202, 138, 9, 4434, 19, 4, 96, 183, 26, 4, 192, 15, 27, 5842, 799, 7101, 39455, 588, 84, 11, 4, 3231, 152, 339, 5206, 42, 4869, 30497, 6293, 345, 4804, 37377, 142, 43, 218, 208, 54, 29, 853, 659, 46, 4, 882, 183, 80, 115, 30, 4, 172, 174, 10, 10, 1001, 398, 1001, 1055, 526, 34, 3717, 68395, 5262, 63370, 17, 4, 6706, 1094, 871, 64, 85, 22, 2030, 1109, 38, 230, 9, 4, 4324, 20636, 251, 5056, 1034, 195, 301, 14, 16, 31, 7, 4, 46035, 8, 783, 48545, 33, 4, 2945, 103, 465, 16454, 42, 845, 45, 446, 11, 1895, 19, 184, 76, 32, 4, 5310, 207, 110, 13, 197, 4, 14906, 16, 601, 964, 2152, 595, 13, 258, 4, 1730, 66, 338, 55, 5312, 4, 550, 728, 65, 1196, 8, 1839, 61, 1546, 42, 8361, 61, 602, 120, 45, 7304, 6, 320, 786, 99, 196, 11100, 786, 5936, 4, 225, 4, 373, 1009, 33, 4,

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by imdb.get_word_index()

In [ ]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print(len(id2word))

train_set = []
test_set = []
for i in range (len(X_train)):
  train_set.append([id2word.get(i, " ") for i in X_train[i]])
for i in range (len(X_test)):
  test_set.append([id2word.get(i, " ") for i in X_test[i]])
print(len(train_set))
print(len(test_set))

**Implementing Bag-of-words model from scratch**

In [ ]:
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer, SnowballStemmer
import gensim
stemmer = SnowballStemmer(language = 'english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# lemmatize_stemming each token and delete all the special characters from each token if exist
import re
def preprocess(token):
    result = " "
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and "'" not in token:
      if token == "wanna":
        token = "want"
      elif token == "gonna":
        token = "going"
      elif token == "gotta":
        token = "got"
      token = re.sub(r'[^\w]', '', token)
      result = (lemmatize_stemming(token))
      #print(result)
    return result

print(preprocess("gonna"))
def refine(train_set):
  result=[]
  for i in range (len(train_set)):
    col = []
    for j in range(len(train_set[i])):
      token = train_set[i][j]
      token = preprocess(token)
      if token != " ":
        col.append(token)
    result.append(col)
  return result
  
train_set = refine(train_set)
test_set = refine(test_set)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
go


In [ ]:
# build sentences again after pruning
def concatenate_list_data(list):
    result = ' '.join(word for word in list)
    return result
 
sentences = []
for i in range (len(train_set)):
  sentences.append(concatenate_list_data(train_set[i]))

test_sentences = []
for i in range (len(test_set)):
  test_sentences.append(concatenate_list_data(test_set[i]))

In [ ]:
# flatten the training set to get the unique words
train_list = []
for sublist in train_set:
    for item in sublist:
        train_list.append(item)
print(len(train_list))

# using set
# function to get unique values
def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return unique_list
        
unique_words = unique(train_list)
print(unique_words)
print(len(unique_words))

1427198
['dutt', 'unachiev', 'contract', 'dispit', 'align', 'kammerud', 'hackensack', 'trevino', 'libr', 'biddi', 'sedari', '1979', 'incess', 'climacter', 'landen', 'finleyson', 'inhuman', 'industrialis', 'winterwond', 'attain', 'cinemat', 'uptak', 'timbo', 'babson', 'transceiv', 'grosbard', 'garrison', 'darkw', 'guzman', 'hyperspac', 'tenness', 'aficionado', 'diction', 'paer', 'sail', 'gratifi', 'grin', 'etho', 'comt', 'dictionari', 'pail', 'scholl', 'apocryph', 'untouch', 'freudian', 'mortenson', 'funfar', 'schoolbus', 'eragorn', '1926', 'dissectionist', 'whini', 'unctuous', 'foliag', 'angelus', 'everyth', 'mcgilli', 'burmes', 'plaid', 'nanavati', 'trough', 'shortag', 'constip', 'pretti', 'psycholog', 'streak', 'kacey', 'subset', 'hujan', 'crumpet', 'appr', 'shambl', 'otoh', 'herbivor', 'justiça', 'loi', 'snub', 'directv', 'grmpfli', 'pochath', 'pych', 'cyanid', 'uppanc', 'moviemi', 'gruel', 'unmanipul', 'mcveigh', 'compassion', 'fopperi', 'ker', 'germna', 'verdon', 'wantabe', 'typog

In [ ]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear.
'''
#Index dictionary to assign an index to each word in vocabulary
from collections import defaultdict 
index_word = {}
i = 0
for word in unique_words:
    index_word[word] = i 
    i += 1

print(sentences[0])
#number of words in the vocab
len_vector = len(unique_words)
nltk.download("punkt")

def bag_of_words(sent):
    count_dict = defaultdict(int)
    vec = np.zeros(len_vector)
    sentence_tokens = nltk.word_tokenize(sent)
    for item in sentence_tokens:
        count_dict[item] += 1
    for key,item in count_dict.items():
        #print(key)
        try:
          vec[index_word[key]] = item
        except:
          print("word isn't in the vocabulary set")
          #print(index_word[item])
    return vec  


power let love reach journalist atmospher room titil heart show year go villaronga help moment chest visual movi current film potenti unfortun camp movi movi scari pratfal stori wonder see charact musician heart shadow critic tricki see land histori half mean boat think frog script histori heart real barrel script arm act watch heartfelt film want
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Creating a frequency matrix on the train, test document sets**


In [ ]:
frequency_matrix = []
for sentence in sentences:
  #print(sentence)
  frequency_matrix.append(bag_of_words(sentence))


test_frequency_matrix = []
for sentence in test_sentences:
  #print(sentence)
  test_frequency_matrix.append(bag_of_words(sentence))

**Train and evaluate the model**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(frequency_matrix, y_train)

y_pred = naive_bayes.predict(test_frequency_matrix)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[892 155]
 [231 722]]
              precision    recall  f1-score   support

           0       0.79      0.85      0.82      1047
           1       0.82      0.76      0.79       953

    accuracy                           0.81      2000
   macro avg       0.81      0.80      0.81      2000
weighted avg       0.81      0.81      0.81      2000



In [ ]:
from sklearn.metrics import accuracy_score
print("the accuracy is {}".format(accuracy_score(y_test, y_pred)))

the accuracy is 0.807


In [ ]:
def test_preprocess(sentence):
    result = " "
    # tokenize the sentence
    for token in gensim.utils.simple_preprocess(sentence) :
      if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and "'" not in token:
        if token == "wanna":
          token = "want"
        elif token == "gonna":
          token = "going"
        elif token == "gotta":
          token = "got"
        token = re.sub(r'[^\w]', '', token)
        result = (lemmatize_stemming(token))
        #print(result)
    return result

# Vectorize text reviews to numbers
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(stop_words='english')
sentence1 = "Very bad movie. I was upset when I watched it and regretted wasting my time. These films should be banned from cinemas"
preprocessed1 = test_preprocess(sentence1)
bags1 = bag_of_words(preprocessed1)
print(naive_bayes.predict(bags1.reshape(1, -1) ))

sentence2 = "I loved this movie because it promotes peace and I think it should compete for an Oscar"
preprocessed2 = test_preprocess(sentence2)
bags2 = bag_of_words(preprocessed2)
print(naive_bayes.predict(bags2.reshape(1, -1) ))

[0]
[1]
